In [ ]:
from langchain_google_genai import  ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import os
topic = ['History','Space','AI']

template = ChatPromptTemplate.from_template("""You are helpfull assistant.Your task is to generate 30 different questions on that topic which is delimited with triple back ticks and sperate each question with newline.Please do your best it is very important to my career.please avoide to generate any extra text excluding question may be it's effect will be irreversible.
Topic: ```{topic}```""")
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",google_api_key=os.getenv("google_api_key")).with_fallbacks([ChatGoogleGenerativeAI(model='gemini-1.5-flash',google_api_key=os.getenv("dgoogle_api_key"))])

chain = template | model | StrOutputParser() | (lambda x:[q for q in x.split('\n') if q])

questions = chain.batch(topic)
# questions = chain.invoke(topic[2])

In [ ]:
questions[0]

In [ ]:
def flatten(lists:list[list]):
    questions = []
    for sublist in lists:
        questions.extend(sublist)

    return questions

_questions = flatten(questions)

In [ ]:
template = ChatPromptTemplate.from_template("""You are helpfull assistant your task is to generate a compherhensive article on that title (which is delimited with triple backtick) please just genrate an article do not add title in the start of aricle pleaes do your best it is very important to my career:
Title: ```{title}```""")

chain2 = template | model | StrOutputParser()

articles = chain2.batch(_questions)

In [ ]:
from langchain_core.documents import Document

Docs = []
for item in zip(articles,_questions):
    Docs.append(Document(page_content=item[0],metadata={"title":item[1]}))

print(Docs)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()
# len(embeddings.embed_query("HI"))

In [ ]:
from langchain_community.vectorstores import Chroma

# vectorstore = Chroma.from_documents(documents=Docs,embedding=embeddings,persist_directory="chroma")

In [ ]:
vectorstore = Chroma(persist_directory="chroma",embedding_function=embeddings).as_retriever(search_kwargs={"k":5})

In [ ]:
# res = vectorstore.invoke("What are the major events in the history of Europe?")
res = vectorstore.invoke("what is space?")
for item in res:
    print(item.page_content)
    print("--" * 60)